from sklearn.model_selection import GridSearchCV

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

## Set grid parameters
###### logistic regression params
    lr_param_grid = {'C': [.001,.001,.01,.1,1,10], 'penalty': [l1,l2]}
###### svc params
    svc_param_grid = {'C': [1,10,100,1000], 'gamma': [.0001,.001,.01], 'kernel': ['linear', 'rbf', 'sigmoid']}
###### decision tree params
    dt_param_grid = {'max_depth': [1,5,10], 'min_samples_split': [.1,1 ,10], 'min_samples_leaf': [.1,.5,5], 'max_features': [2,6,10]}
###### xgb params
    xgb_param_grid = {"learning_rate": [.01,0.1,1],'max_depth': [2,6,10],'min_child_weight': [5,10,15],'n_estimators': [50, 100, 250]}
###### adaboost params
    ab_param_grid = {"learning_rate": [.01,0.1,1],'algorithm': ['SAMME', 'SAMME.R'],'n_estimators': [50, 100, 250]}
###### gradient boost params
    gb_param_grid = {'learning_rate': [.01,0.1,1],'max_depth': [2,6,10],'min_samples_leaf': [1,3,6],'min_samples_split': [1,2,4],'n_estimators': [50, 100, 250]}
###### random forest params
    rf_param_grid = {'max_depth': [2,6,10],'min_samples_leaf': [1,3,6],'min_samples_split': [1,2,4],'n_estimators': [5, 10, 25]}

## GridSearchCV

Steps:
- determine estimator
- define the hyperparameters to tune
- define a range of values of your hyperparameters
- decide cross-validation scheme
- define a score function
- include useful info. or functions
- setting 'refit' parameter = True allows direct use of the new parameters to fit the data

        # Create a Random Forest Classifier with specified criterion
        rf_class = RandomForestClassifier(criterion='entropy')

        # Create the parameter grid
        param_grid = {'max_depth': [2, 4, 8, 15], 'max_features': ['auto', 'sqrt']} 

        # Create a GridSearchCV object
        grid_rf_class = GridSearchCV(
            estimator=rf_class,
            param_grid=param_grid,
            scoring='roc_auc',
            n_jobs=4,
            cv=5,
            refit=True, return_train_score=True)
        print(grid_rf_class)
        
### CV properties     

    # Read the cv_results property into a dataframe & print it out
    cv_results_df = pd.DataFrame(grid_rf_class.cv_results_)
    print(cv_results_df)

    # Extract and print the column with a dictionary of hyperparameters used
    column = cv_results_df.loc[:, ['params']]
    print(column)

    # Extract and print the row that had the best mean test score
    best_row = cv_results_df[cv_results_df['rank_test_score'] == 1 ]
    print(best_row)
    
    # Print out the ROC_AUC score from the best-performing square
    best_score = grid_rf_class.best_score_
    print(best_score)

    # Create a variable from the row related to the best-performing square
    cv_results_df = pd.DataFrame(grid_rf_class.cv_results_)
    best_row = cv_results_df.loc[[grid_rf_class.best_index_]]
    print(best_row)

    # Get the n_estimators parameter from the best-performing square and print
    best_n_estimators = grid_rf_class.best_params_["n_estimators"]
    print(best_n_estimators)
    
       # See what type of object the best_estimator_ property is
    print(type(grid_rf_class.best_estimator_))

    # Create an array of predictions directly using the best_estimator_ property
    predictions = grid_rf_class.best_estimator_.predict(X_test)

    # Take a look to confirm it worked, this should be an array of 1's and 0's
    print(predictions[0:5])

    # Now create a confusion matrix 
    print("Confusion Matrix \n", confusion_matrix(y_test, predictions))

    # Get the ROC-AUC score
    predictions_proba = grid_rf_class.best_estimator_.predict_proba(X_test)[:,1]
    print("ROC-AUC Score \n", roc_auc_score(y_test, predictions_proba)) 

## RandomizedSearchCV
- similar to GridSearch, but takes a range of values and the number of iterations is set, rather than taking every combination
- parameter differences
    - n_iter = # of random iterations
    - param_distributions = same as param_grid


            # Create the parameter grid
            param_grid = {'learning_rate': np.linspace(0.1,2,150), 'min_samples_leaf': list(range(20,65))} 

            # Create a random search object
            random_GBM_class = RandomizedSearchCV(
                estimator = GradientBoostingClassifier(),
                param_distributions = param_grid,
                n_iter = 10,
                scoring='accuracy', n_jobs=4, cv = 5, refit=True, return_train_score = True)

            # Fit to the training data
            random_GBM_class.fit(X_train, y_train)

### Properties          
           
           # Print the values used for both hyperparameters
            print(random_GBM_class.cv_results_['param_learning_rate'])
            print(random_GBM_class.cv_results_['param_min_samples_leaf'])

## Informed Search
- sequential model building and learning

### coarse to fine tuning
- random search
- review results
- grid search in the smaller area (could also be more random searches)
- continue until optimal score obtained

### bayesian tuning
- pick hyperparameter combination
- build model
- get new evidence
- update beliefs and chose better hyperparameters next round
- popular for larger tasks
- Hyperopt package
    - needs uniform distributions, so a hp.quniform or hp.uniform is recommended
    - an objective function needs to be created to calculate loss per iteration

            
            from hyperopt import hp, space, tpe, fmin
            # Set up space dictionary with specified hyperparameters
            space = {'max_depth': hp.quniform('max_depth', 2, 10, 2),'learning_rate': hp.uniform('learning_rate', 0.001,0.9)}

            # Set up objective function
            def objective(params):
                params = {'max_depth': int(params['max_depth']),'learning_rate': params['learning_rate']}
                gbm_clf = GradientBoostingClassifier(n_estimators=100, **params) 
                best_score = cross_val_score(gbm_clf, X_train, y_train, scoring='accuracy', cv=2, n_jobs=4).mean()
                loss = 1 - best_score
                return loss

            # Run the algorithm
            best = fmin(fn=objective,space=space, max_evals=20, rstate=np.random.RandomState(42), algo=tpe.suggest)
            print(best)
            
### genetic algorithms
- create some models
- pick the best by scoring function
- create new models similar to the best ones
- add randomness so local optimum isn't reached
- repeat
- TPOT library
    - generations - # of cycles
    - population_size = # of models to keep
    - offspring_size = # of offspring in each 
    - mutation_rate = proportion of pipelines to apply randomness
    - crossover_rate = proportion of pipelines to breed each iteration
    - scoring = objective function
    - cv = cross validation
    
            # Assign the values outlined to the inputs
            number_generations = 3
            population_size = 4
            offspring_size = 3
            scoring_function = 'accuracy'

            # Create the tpot classifier
            tpot_clf = TPOTClassifier(generations=number_generations, population_size=population_size,
                                      offspring_size=offspring_size, scoring=scoring_function,
                                      verbosity=2, random_state=2, cv=2)

            # Fit the classifier to the training data
            tpot_clf.fit(X_train, y_train)

            # Score on the test set
            print(tpot_clf.score(X_test, y_test))

## Hyperparamter Visualizations for Classification

#### Max Tree Depth
    max_depths = np.linspace(1, 32, 32, endpoint=True)
    train_results = []
    test_results = []
    for max_depth in max_depths:
       dt = DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
       dt.fit(x_train, y_train)
       train_pred = dt.predict(x_train)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       ###### Add auc score to previous train results
       train_results.append(roc_auc)
       y_pred = dt.predict(x_test)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       ###### Add auc score to previous test results
       test_results.append(roc_auc)
    plt.figure(figsize=(12,6))
    plt.plot(max_depths, train_results, 'b', label='Train AUC')
    plt.plot(max_depths, test_results, 'r', label='Test AUC')
    plt.ylabel('AUC score')
    plt.xlabel('Tree depth')
    plt.legend()
    plt.show()
#### Minimum Sample Split
    min_samples_splits = np.linspace(0.1, 1.0, 10, endpoint=True)
    train_results = []
    test_results = []
    for min_samples_split in min_samples_splits:
       dt = DecisionTreeClassifier(criterion='entropy', min_samples_split=min_samples_split)
       dt.fit(x_train, y_train)
       train_pred = dt.predict(x_train)
       false_positive_rate, true_positive_rate, thresholds =    roc_curve(y_train, train_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       train_results.append(roc_auc)
       y_pred = dt.predict(x_test)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       test_results.append(roc_auc)
    plt.figure(figsize=(12,6))
    plt.plot(min_samples_splits, train_results, 'b', label='Train AUC')
    plt.plot(min_samples_splits, test_results, 'r', label='Test AUC')
    plt.xlabel('Min. Sample splits')
    plt.legend()
    plt.show()
#### Minimum Sample Leaf
    min_samples_leafs = np.linspace(0.1, 0.5, 5, endpoint=True)
    train_results = []
    test_results = []
    for min_samples_leaf in min_samples_leafs:
       dt = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=min_samples_leaf)
       dt.fit(x_train, y_train)
       train_pred = dt.predict(x_train)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       train_results.append(roc_auc)
       y_pred = dt.predict(x_test)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       test_results.append(roc_auc)
    plt.figure(figsize=(12,6))    
    plt.plot(min_samples_leafs, train_results, 'b', label='Train AUC')
    plt.plot(min_samples_leafs, test_results, 'r', label='Test AUC')
    plt.ylabel('AUC score')
    plt.xlabel('Min. Sample Leafs')
    plt.legend()
    plt.show()
#### Maximum Features
    max_features = list(range(1,x_train.shape[1]))
    train_results = []
    test_results = []
    for max_feature in max_features:
       dt = DecisionTreeClassifier(criterion='entropy', max_features=max_feature)
       dt.fit(x_train, y_train)
       train_pred = dt.predict(x_train)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       train_results.append(roc_auc)
       y_pred = dt.predict(x_test)
       false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
       roc_auc = auc(false_positive_rate, true_positive_rate)
       test_results.append(roc_auc)
    plt.figure(figsize=(12,6))
    plt.plot(max_features, train_results, 'b', label='Train AUC')
    plt.plot(max_features, test_results, 'r', label='Test AUC')
    plt.ylabel('AUC score')
    plt.xlabel('max features')
    plt.legend()
    plt.show()
#### Tree Depth (Regressor using R2 and MSE)
    max_depths = np.linspace(1, 30, 30, endpoint=True)
    mse_results = []
    r2_results = []
    for max_depth in max_depths:
        regressor = DecisionTreeRegressor(max_depth=max_depth, random_state=45)
        regressor.fit(x_train, y_train)
        y_pred = regressor.predict(x_test)
        score = performance(y_test, y_pred)
        r2_results.append(score[0])
        mse_results.append(score[1])
    plt.figure(figsize=(12,6))
    plt.plot(max_depths, r2_results, 'b', label='R2')
    plt.xlabel('Tree Depth')
    plt.ylabel('R-squared')
    plt.legend()
    plt.show()
    plt.figure(figsize=(12,6))
    plt.plot(max_depths, mse_results, 'r', label='MSE')
    plt.xlabel('Tree Depth')
    plt.ylabel('MSE')
    plt.legend()
    plt.show()